<a href="https://colab.research.google.com/github/sarkar-ai-26/Ai_Project/blob/main/LTTS_TestGuide_TestAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate bitsandbytes sentencepiece huggingface_hub

In [ ]:
# STEP 2: Authenticate Hugging Face (LLaMA 2 access required)
# -----------------------------------------------
from huggingface_hub import login

# 🔐 Replace with your Hugging Face token (must have access to LLaMA 2)
login(token='')  # <-- PUT YOUR TOKEN HERE

In [ ]:

# -----------------------------------------------
# STEP 3: Load LLaMA 2 (7B Chat) model from Hugging Face
# -----------------------------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "meta-llama/Llama-2-7b-chat-hf"  # LLaMA 2 7B Chat version

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Load model (≈13GB download, GPU required)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # Uses GPU if available
    torch_dtype=torch.float16,  # Faster and lighter
    trust_remote_code=True
)

# -----------------------------------------------
# STEP 4: Create a generation pipeline
# -----------------------------------------------
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.3,
    repetition_penalty=1.1
)

# Optional test
print("✅ Model is ready. Sample response:")
print(llama_pipeline("### Human: What is software testing?\n### Assistant:")[0]["generated_text"])

# -----------------------------------------------
# STEP 5: Save model + tokenizer locally for reuse
# -----------------------------------------------
model_path = "./llama2-7b-chat-hf"

# Save locally (for reuse or API deployment)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print(f"\n📦 Model & tokenizer saved at: {model_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cpu


✅ Model is ready. Sample response:


In [ ]:
import pandas as pd

# STEP 1: Mount Google Drive (uncomment if your file is in Drive)
# from google.colab import drive
# drive.mount('/content/drive')

# STEP 2: Path to your Excel file
file_path = "result.xlsx"  # update path if needed

# STEP 3: Load all sheet names
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

# STEP 4: Define keywords to filter relevant sheets
relevant_keywords = ['test', 'export', 'report', 'log', 'result']
filtered_sheets = [
    sheet for sheet in sheet_names
    if any(keyword in sheet.lower() for keyword in relevant_keywords)
]

# STEP 5: Extract test result summary from each relevant sheet
summary_data = []

for sheet in filtered_sheets:
    try:
        df = pd.read_excel(file_path, sheet_name=sheet, header=1)

        # Detect verdict and step columns
        verdict_col = next((col for col in df.columns if 'verdict' in str(col).lower()), None)
        step_col = df.columns[0]  # usually step name

        if verdict_col is None:
            continue

        df = df.dropna(subset=[verdict_col])
        total_tests = len(df)
        verdict_counts = df[verdict_col].value_counts().to_dict()

        failed_steps = (
            df[df[verdict_col].astype(str).str.upper() == 'FAILED'][step_col]
            .value_counts()
            .head(3)
            .to_dict()
        )

        summary_data.append({
            "Sheet": sheet,
            "Total Tests": total_tests,
            "Verdict Counts": verdict_counts,
            "Top Failed Steps": failed_steps
        })

    except Exception as e:
        print(f"❌ Skipping sheet '{sheet}': {e}")
        continue

# STEP 6: Display the extracted summary
summary_df = pd.DataFrame(summary_data)


In [ ]:

# Function to format summary DataFrame into an LLM-friendly prompt
def generate_llm_prompt(summary_df):
    prompt = "📊 You are a QA Analyst. Here is the summary of the Excel-based test report:\n\n"

    for _, row in summary_df.iterrows():
        prompt += f"🗂️ Sheet: {row['Sheet']}\n"
        prompt += f"🔢 Total Test Cases: {row['Total Tests']}\n"

        prompt += "✅ Verdict Breakdown:\n"
        for verdict, count in row['Verdict Counts'].items():
            prompt += f"   - {verdict}: {count}\n"

        if row['Top Failed Steps']:
            prompt += "⚠️ Top 3 Failed Steps:\n"
            for step, count in row['Top Failed Steps'].items():
                prompt += f"   - {step}: {count} failures\n"

        prompt += "\n"

    prompt += (
        "Please analyze the data and provide insights:\n"
        "- What patterns do you observe in the test failures?\n"
        "- Are there specific areas or steps that need improvement?\n"
        "- Provide a short summary and suggestions to improve test quality.\n"
    )

    return prompt


# Generate prompt from the summary DataFrame
llm_prompt = generate_llm_prompt(summary_df)

# Display the final prompt
print(llm_prompt[:2000])  # Preview first 2000 characters
